# Machine Learning Aplicado con Python

Este proyecto tiene como objetivo utilizar distintas librerías de Python (Scikit-learn, pandas, numpy, matplotlib, etc...) para entender todo el flujo de trabajo que se realiza cuando se resuelve un problema de Aprendizaje Supervisado de Machine Learning. En este caso, se busca predecir los ingresos que tendrá una película en base a un histórico de datos de películas de la plataforma imdb.

Otras disciplinas:
- Feature engineering
- Selección de modelos

## Antes de la Práctica:

***Hay que reconocer el tipo de aprendizaje que se necesita***

1. ¿Qué beneficio se piensa generar y para quién?  
2. ¿Cuál de las siguientes funcionalidades sería más útil para lograr ese objetivo?:  
        A. Predecir una métrica  
        B. Predecir una etiqueta  
        C. Agrupar elementos similares  
        D. Optimizar un proceso con prueba y error  
    
Los dos primeros corresponden a **Aprendizaje supervisado**.  

***Hay que aterrizar el problema de aprendizaje supervisado***.  

1. ¿El valor que se quiere predecir es de qué tipo?  
        A. Continuo
        B. Discreto
2. ¿Cuál es la definición de éxito de la predicción?
3. ¿Con qué datos se contará para hacer la predicción?
4. ¿A qué disciplina en particular pertenece la pregunta que se está tratando de resolver?
5. Considerando la intucipon sobre la disciplina, ¿Pueden los datos permitir predecir el objetivo?

### Contexto del Problema

Contexto: Un *ente gubernamental* quiere definir sus políticas de financiamiento de producciones cinematográficas nacionales.

**Sus objetivos son:**  
1. **Ayudar** a la producción de películas de calidad **que no logran ser autosustentables**  
2. Saber qué películas tienen más dificultad de recuperar lo gastado en el presupuesto.  

Por consiguiente, el ente gubernamental quiere **predecir una métrica**: El ingreso mundial generado por una película.  

### Desarrollo del Problema

Estando ubicados en el contexto del problema, se podrían responder las anteriores preguntas de la siguiente manera:

1. Los ingresos de una película corresponden a **valores continuos**
2. El éxito será "**qué tan cerca se está del valor real de ingreso generado por la película**"
3. Los datos se recogerán de **bases de datos públicas de internet**
4. El dominio de trabajo es la **industria del cine**, en particular de la distribución de películas.
5. Los datos podrán permitir predecir el objetivo. De forma general existen características que permiten determinar si una película será exito o no, como: **calidad, actores, presupuesto, etc...**

### Preparación de datos

***NumPy***: librería de matriz n-dimensional base  
***Pandas***: estructura de datos y análisis  
***Matplotlib***: trazado completo 2D  
***Seaborn***: visualización estadística de datos  

In [1]:
# Importamos las librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

**%matplotlib inline** permite que los gráficos estén renderizados directamente en el Notebook.

In [2]:
%matplotlib inline

Se cargan los datos desde un github que contiene un archivo csv.
Dicho archivo se convierte a un DataFrame de pandas.

In [3]:
url = 'https://github.com/JuanPabloMF/datasets-platzi-course/raw/master/datasets/peliculas.csv'
movies_df = pd.read_csv(url, encoding='utf-8')

#### Columnas del DataFrame

Variables predictoras:  
**movie_title**: *Título de la película*  
**title_year**: *Año de estreno de la película*  
**duration**: *Duración de la película en minutos*  
**language**: *Lenguaje original de la película*  
**country**: *País que produció la película*  
**genres**: *Género(s) de la película*  
**content_rating**: *Para qué público va dirigida la película*  
**plot_keywords**: *Palabras clave que definen la película*  
**director_name**: *Nombre del director principal de la película*  
**actor_1_name, actor_2_name, actor_3_name**: *Actores principales que actúan en la película*  
**cast_total_facebook_likes**: *Número total de likes que tiene el casting en Facebook*  
**budget**: *Cuánto presupuesto tiene la película*  
**imdb_score**: *Qué score tiene la película en imdb*  

Variable objetivo:  
**gross**: *Cuánto generó de ingreso la película*  

In [4]:
# se visualizan los primeros registros del DataFrame
movies_df.head()

,movie_title,title_year,color,aspect_ratio,duration,language,country,duration.1,genres,content_rating,plot_keywords,director_name,actor_1_name,actor_2_name,actor_3_name,cast_total_facebook_likes,budget,imdb_score,gross
0,Avatar,2009.0,Color,1.78,178.0,English,USA,178.0,Action|Adventure|Fantasy|Sci-Fi,PG-13,avatar|future|marine|native|paraplegic,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,4834,237000000.0,7.9,760505847.0
1,Pirates of the Caribbean: At World's End,2007.0,Color,2.35,169.0,English,USA,169.0,Action|Adventure|Fantasy,PG-13,goddess|marriage ceremony|marriage proposal|pi...,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,48350,300000000.0,7.1,309404152.0
2,Spectre,2015.0,Color,2.35,148.0,English,UK,148.0,Action|Adventure|Thriller,PG-13,bomb|espionage|sequel|spy|terrorist,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11700,245000000.0,6.8,200074175.0
3,The Dark Knight Rises,2012.0,Color,2.35,164.0,English,USA,164.0,Action|Thriller,PG-13,deception|imprisonment|lawlessness|police offi...,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,106759,250000000.0,8.5,448130642.0
4,Star Wars: Episode VII - The Force Awakens ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Documentary,NaN,NaN,Doug Walker,Doug Walker,Rob Walker,NaN,143,NaN,7.1,NaN


In [5]:
# visualización de las columnas
movies_df.columns

Index(['movie_title', 'title_year', 'color', 'aspect_ratio', 'duration',
       'language', 'country', 'duration.1', 'genres', 'content_rating',
       'plot_keywords', 'director_name', 'actor_1_name', 'actor_2_name',
       'actor_3_name', 'cast_total_facebook_likes', 'budget', 'imdb_score',
       'gross'],
      dtype='object')

## Entendimiento de los datos

In [6]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   movie_title                5043 non-null   object 
 1   title_year                 4935 non-null   float64
 2   color                      5024 non-null   object 
 3   aspect_ratio               4714 non-null   float64
 4   duration                   5028 non-null   float64
 5   language                   5031 non-null   object 
 6   country                    5038 non-null   object 
 7   duration.1                 5028 non-null   float64
 8   genres                     5043 non-null   object 
 9   content_rating             4740 non-null   object 
 10  plot_keywords              4890 non-null   object 
 11  director_name              4939 non-null   object 
 12  actor_1_name               5036 non-null   object 
 13  actor_2_name               5030 non-null   objec

In [7]:
movies_df.dtypes

movie_title                   object
title_year                   float64
color                         object
aspect_ratio                 float64
duration                     float64
language                      object
country                       object
duration.1                   float64
genres                        object
content_rating                object
plot_keywords                 object
director_name                 object
actor_1_name                  object
actor_2_name                  object
actor_3_name                  object
cast_total_facebook_likes      int64
budget                       float64
imdb_score                   float64
gross                        float64
dtype: object

#### Separación de datos

In [8]:
# Obtención de columnas numéricas
num = (movies_df.dtypes == 'float64') | (movies_df.dtypes == 'int64')
num

movie_title                  False
title_year                    True
color                        False
aspect_ratio                  True
duration                      True
language                     False
country                      False
duration.1                    True
genres                       False
content_rating               False
plot_keywords                False
director_name                False
actor_1_name                 False
actor_2_name                 False
actor_3_name                 False
cast_total_facebook_likes     True
budget                        True
imdb_score                    True
gross                         True
dtype: bool

In [9]:
print(num.index)
print(num['movie_title'])

Index(['movie_title', 'title_year', 'color', 'aspect_ratio', 'duration',
       'language', 'country', 'duration.1', 'genres', 'content_rating',
       'plot_keywords', 'director_name', 'actor_1_name', 'actor_2_name',
       'actor_3_name', 'cast_total_facebook_likes', 'budget', 'imdb_score',
       'gross'],
      dtype='object')
False


In [10]:
num_cols = [column for column in num.index if num[column]]
num_cols

['title_year',
 'aspect_ratio',
 'duration',
 'duration.1',
 'cast_total_facebook_likes',
 'budget',
 'imdb_score',
 'gross']

In [13]:
# Obtención de columnas de texto (object)
obj = movies_df.dtypes == object
obj_cols = [column for column in obj.index if obj[column]]

In [14]:
obj_cols

['movie_title',
 'color',
 'language',
 'country',
 'genres',
 'content_rating',
 'plot_keywords',
 'director_name',
 'actor_1_name',
 'actor_2_name',
 'actor_3_name']